# TensorFlow自动求导机制

 TensorFlow的自动求导机制

 Variable对象
 
 对Tensor对象的进一步封装

在模型训练过程中自动记录梯度信息，由算法自动优化

可以被训练的变量

在机器学习中作为模型参数

tf.Variable(initial_value,dtype)

Variable可以是数字、Python列表、nd_array对象、Tensor对象


### 创建可训练变量

In [1]:
import tensorflow as tf
print("TensorFlow version:",tf.__version__)

TensorFlow version: 2.5.0-rc3


In [2]:
import numpy as np

In [3]:
tf.Variable(3)#Variable初始值为3

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=3>

In [4]:
tf.Variable([1,2])#Variable初始值为[1,2]

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([1, 2])>

In [5]:
tf.Variable(np.array([1,2]))

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([1, 2])>

In [6]:
tf.Variable(3.)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=3.0>

In [7]:
tf.Variable([1,2],dtype=tf.float64)

<tf.Variable 'Variable:0' shape=(2,) dtype=float64, numpy=array([1., 2.])>

### 将张量封装为可训练变量

In [8]:
tf.Variable(tf.constant([[1,2],[3,4]]))

<tf.Variable 'Variable:0' shape=(2, 2) dtype=int32, numpy=
array([[1, 2],
       [3, 4]])>

In [9]:
tf.Variable(tf.zeros([2,3]))

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[0., 0., 0.],
       [0., 0., 0.]], dtype=float32)>

In [10]:
tf.Variable(tf.random.normal([2,2]))

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[-0.80655473, -0.10838908],
       [-0.03100266,  1.8767376 ]], dtype=float32)>

In [11]:
x=tf.Variable([1,2])

In [12]:
x

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([1, 2])>

In [13]:
print(x.shape,x.dtype)

(2,) <dtype: 'int32'>


In [14]:
print(x.numpy())

[1 2]


In [15]:
x.trainable

True

In [16]:
type(x)

tensorflow.python.ops.resource_variable_ops.ResourceVariable

### 可训练变量赋值

In [17]:
x=tf.Variable([1,2])

In [18]:
x.assign([3,4])

<tf.Variable 'UnreadVariable' shape=(2,) dtype=int32, numpy=array([3, 4])>

In [19]:
x.assign_add([1,1])

<tf.Variable 'UnreadVariable' shape=(2,) dtype=int32, numpy=array([4, 5])>

In [20]:
x.assign_add([1,1])

<tf.Variable 'UnreadVariable' shape=(2,) dtype=int32, numpy=array([5, 6])>

In [21]:
a=tf.constant(2)

In [22]:
#a.assign(3)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

In [23]:
#a.trainable

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'trainable'

In [24]:
a=tf.range(5)

In [25]:
x=tf.Variable(a)

In [26]:
isinstance(a,tf.Tensor),isinstance(a,tf.Variable)

(True, False)

In [27]:
isinstance(x,tf.Tensor),isinstance(x,tf.Variable)

(False, True)

### 自动求导——GradientTape

with GradientTape() 

函数表达式 

grad=tape.gradient(函数,自变量

In [28]:
x=tf.Variable(3.)

In [29]:
with tf.GradientTape() as tape:
    y=tf.square(x)

In [30]:
dy_dx=tape.gradient(y,x)

In [31]:
print(y)
print(dy_dx)

tf.Tensor(9.0, shape=(), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)


GradientTape(persistent, watch_accessed_variables)

In [33]:
x=tf.Variable(3.)

with tf.GradientTape(persistent=True) as tape:
    y=tf.square(x)
    z=pow(x,3)
    
dy_dx=tape.gradient(y,x)
dz_dx=tape.gradient(z,x)

print(y)
print(dy_dx)
print(z)
print(dz_dx)

del tape

tf.Tensor(9.0, shape=(), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)
tf.Tensor(27.0, shape=(), dtype=float32)
tf.Tensor(27.0, shape=(), dtype=float32)


In [34]:
x=tf.Variable(3.)

with tf.GradientTape(watch_accessed_variables=False) as tape:
    y=tf.square(x)
    
dy_dx=tape.gradient(y,x)

print(y)
print(dy_dx)

tf.Tensor(9.0, shape=(), dtype=float32)
None


### 添加监视——watch（）

In [36]:
x=tf.Variable(3.)

with tf.GradientTape(watch_accessed_variables=False) as tape:
    tape.watch(x)
    y=tf.square(x)
    
dy_dx=tape.gradient(y,x)

print(y)
print(dy_dx)

tf.Tensor(9.0, shape=(), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)


### 监视非可训练对象

In [39]:
x=tf.constant(3.)

with tf.GradientTape(watch_accessed_variables=False) as tape:
    tape.watch(x)
    y=tf.square(x)
    
dy_dx=tape.gradient(y,x)

print(y)
print(dy_dx)

tf.Tensor(9.0, shape=(), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)


### 多元函数求偏导

In [37]:
x=tf.Variable(3.)
y=tf.Variable(4.)

with tf.GradientTape() as tape:
    f=tf.square(x)+2*tf.square(y)+1

df_dx,df_dy=tape.gradient(f,[x,y])

print(f)
print(df_dx)
print(df_dy)

tf.Tensor(42.0, shape=(), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)
tf.Tensor(16.0, shape=(), dtype=float32)


In [38]:
x=tf.Variable(3.)
y=tf.Variable(4.)

with tf.GradientTape() as tape:
    f=tf.square(x)+2*tf.square(y)+1

first_grads=tape.gradient(f,[x,y])#用一个变量求两个偏导

print(first_grads)

[<tf.Tensor: shape=(), dtype=float32, numpy=6.0>, <tf.Tensor: shape=(), dtype=float32, numpy=16.0>]


In [41]:
x=tf.Variable(3.)
y=tf.Variable(4.)

with tf.GradientTape(persistent=True) as tape:
    f=tf.square(x)+2*tf.square(y)+1

df_dx=tape.gradient(f,x)
df_dy=tape.gradient(f,y)#分别求偏导

print(f)
print(df_dx)
print(df_dy)

del tape#释放tape

tf.Tensor(42.0, shape=(), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)
tf.Tensor(16.0, shape=(), dtype=float32)


### 求二阶导数

In [42]:
x=tf.Variable(3.)
y=tf.Variable(4.)

with tf.GradientTape(persistent=True) as tape2:
    
    with tf.GradientTape(persistent=True) as tape1:
        f=tf.square(x)+2*tf.square(y)+1
        
    first_grads=tape1.gradient(f,[x,y])

second_grads=[tape2.gradient(first_grads,[x,y])]

print(f)
print(first_grads)
print(second_grads)

del tape1
del tape2

tf.Tensor(42.0, shape=(), dtype=float32)
[<tf.Tensor: shape=(), dtype=float32, numpy=6.0>, <tf.Tensor: shape=(), dtype=float32, numpy=16.0>]
[[<tf.Tensor: shape=(), dtype=float32, numpy=2.0>, <tf.Tensor: shape=(), dtype=float32, numpy=4.0>]]


### 对向量求偏导

In [43]:
x=tf.Variable([1.,2.,3.])
y=tf.Variable([4.,5.,6.])
with tf.GradientTape() as tape:
    f=tf.square(x)+2*tf.square(y)+1

df_dx,df_dy=tape.gradient(f,[x,y])

print(f)
print(df_dx)
print(df_dy)

tf.Tensor([34. 55. 82.], shape=(3,), dtype=float32)
tf.Tensor([2. 4. 6.], shape=(3,), dtype=float32)
tf.Tensor([16. 20. 24.], shape=(3,), dtype=float32)
